# Standard Name Table for PIV

In [1]:
from ssnolib import parse_table, StandardNameTable, AgentRole, StandardName, VectorStandardName
from ssnolib.prov import Organization, Person
from datetime import datetime
import ontolutils

In [2]:
zenodo_id = 14175299

In [3]:
VERSION = "1.1.0"

In [4]:
description = """This table provides standard names for the description of PIV data. It is an initial version covering the most basic properties of planar PIV data."""

In [18]:
changelog = """
- Improved some standard name descriptions
"""

In [5]:
snt = StandardNameTable(
    title="Standard Name Table for Particle Image Velociemtry data",
    version=VERSION,
    description=description,
    id=f"https://doi.org/10.5281/zenodo.{zenodo_id}",
    # editorial_note="Changed blank nodes to explicit IRIs. This especially makes the standard names unique for the version and allows better visualization in tools like GraphDB. The IRIs uses the Zenodo Record URL as a prefix.",
    created="2024-11-17",
    modified=datetime.today().strftime("%Y-%m-%d")
)
snt

StandardNameTable(id=https://doi.org/10.5281/zenodo.14175299, title=Standard Name Table for Particle Image Velociemtry data, hasVersion=1.0.0, description=This table provides standard names for the description of PIV data. It is an initial version covering the most basic properties of planar PIV data., created=2024-11-17, modified=2024-11-23, standardNames=[])

In [6]:
with ontolutils.set_config(blank_node_prefix_name="local:"):
    KIT = Organization(id="https://ror.org/04t3en479",
                       name="Karlsruhe Institute of Technology, Institute of Thermal Turbomachinery",
                       ror_id="https://ror.org/04t3en479")
    author1 = Person(id="https://orcid.org/0000-0001-8729-0482",
                     first_name="Matthias",
                     last_name="Probst",
                     orcid_id="https://orcid.org/0000-0001-8729-0482",
                    affiliation=KIT)
    author2 = Person(id="https://orcid.org/0000-0001-9560-500X",
                     first_name="Balazs",
                     last_name="Pritz",
                     orcid_id="https://orcid.org/0000-0001-9560-500X",
                     affiliation=KIT)

In [7]:
with ontolutils.set_config(blank_node_prefix_name="local:"):
    snt.add_author(author1, AgentRole.Contact_Person)
    snt.add_author(author2, AgentRole.Supervisor)

## Adding Standard names

In [8]:
import pathlib
from typing import Union

import yaml


def read_standard_names_from_yaml(yaml_filename: Union[str, pathlib.Path]):
    yaml_filename = pathlib.Path(yaml_filename)
    if not yaml_filename.exists():
        raise FileNotFoundError(f"File {yaml_filename} does not exist")

    with open(yaml_filename, 'r') as f:
        data = yaml.safe_load(f)

    standard_names = []
    for k, v in data.items():
        standard_names.append(StandardName(standard_name=k, unit=v["unit"], description=v["description"]))

    return standard_names

In [9]:
with ontolutils.set_config(blank_node_prefix_name="local:"):
    standard_names = read_standard_names_from_yaml("piv_standard_names.yaml")

In [10]:
snt.standardNames = standard_names

In [11]:
json_ld_filename = snt.to_jsonld(
    f"{snt.title.replace(' ', '_')}.jsonld",
    context={"local": f"https://doi.org/10.5281/zenodo.{zenodo_id}/"},
    overwrite=True)
ttl_filename = snt.to_ttl(
    json_ld_filename.with_suffix(".ttl"),
    context={"local": f"https://doi.org/10.5281/zenodo.{zenodo_id}/"},
    overwrite=True
)
xml_filename = snt.to_xml(
    json_ld_filename.with_suffix(".xml"),
    context={"local": f"https://doi.org/10.5281/zenodo.{zenodo_id}/"},
    overwrite=True
)

In [12]:
from h5rdmtoolbox.repository import zenodo
from h5rdmtoolbox.repository.zenodo import metadata
from datetime import datetime

In [13]:
zenodo_id

14175299

In [14]:
real_repo = zenodo.ZenodoRecord(source=zenodo_id, sandbox=False, env_name_for_token="ZENODO_API_TOKEN2")
zen = real_repo

In [15]:
meta = metadata.Metadata(
    version=VERSION,
    title=json_ld_filename.stem.replace("_", " "),
    description=snt.description,
    creators=[metadata.Creator(name=f"{author1.lastName}, {author1.firstName}",
                               affiliation=KIT.name,
                               orcid=author1.orcidId),
             metadata.Creator(name=f"{author2.lastName}, {author2.firstName}",
                                       affiliation=KIT.name,
                                       orcid=author2.orcidId)],
    upload_type='other',
    access_right='open',
    keywords=['standard name table', 'piv', 'particle image velocimetry'],
    publication_date=datetime.now(),
)

In [16]:
zen.set_metadata(meta)

In [17]:
zen.upload_file(json_ld_filename, overwrite=True)
zen.upload_file(ttl_filename, overwrite=True)
zen.upload_file(xml_filename, overwrite=True)